
# 🧬✨ Apical-out Cell Classifier Codelab ✨🔬

Welcome, scientist-friend!  
This tutorial will walk you through how to detect Apical-out cells in microscopy images using pure **Python magic** 🧤🔍

It’s fast. It’s pretty. It even draws in blue 💙 and yellow 💛.  
Let’s gooo 💻🎨🔬

---

## 🌈 What You'll Build

You’ll create a cozy little image analysis tool that:

1. 📸 Loads TIFF microscopy images  
2. 🧼 Cleans and segments cells  
3. 🔵 Identifies Apical-out cells based on intensity & shape  
4. ✨ Draws on the image (Blue = Apical-out! Yellow = Convex hull!)  
5. 📊 Saves results as a neat Excel file on your Desktop


## 📦 Chapter 1: Installation
This chapter will walk you through setting up your Python environment to run the Apical-in/Apical-out image analysis pipeline. By the end of this chapter, you will have all required packages installed and the project ready to execute.

---

### 1.1 Prerequisites

Before beginning, make sure you have the following installed:

- Python 3.8+

You can check your Python version by running:

In [ ]:
python3 --version

### 1.2 (Optional but Recommended) Create a Virtual Environment

In [ ]:
python3 -m venv apical_env
source apical_env/bin/activate  # On macOS/Linux
.\apical_env\Scripts\activate   # On Windows

### 1.3 Project Structure

Make sure your project directory includes the following files:

In [ ]:
project/
│
├── data/                ← Place all .tif images here
├── results/             ← Output folder for Excel and visualizations
├── parameters.py
├── main.py
├── image_processing.py
└── your_notebook.ipynb

### 1.4 Install Required Libraries

Option 1: Use `requirements.txt` (if available):

In [ ]:
pip install -r requirements.txt

Option 2: Manually install each package one by one:

pip install numpy pandas matplotlib opencv-python scikit-image scipy openpyxl

In [ ]:
pip install numpy pandas matplotlib opencv-python scikit-image scipy openpyxl


### 📦 Explanation of Each Package

| Package           | Purpose                                                         |
|------------------|-----------------------------------------------------------------|
| `numpy`          | Numerical computations and array handling                       |
| `pandas`         | Tabular data manipulation and Excel export                      |
| `matplotlib`     | Visualization and plotting                                      |
| `opencv-python` (`cv2`) | Image reading, resizing, and processing               |
| `scikit-image`   | Advanced image analysis (e.g., filtering, labeling, morphology) |
| `scipy`          | Convex hulls, distance transforms, and filtering                |
| `openpyxl`       | Saving results to Excel (.xlsx) files                           |
| `os`             | Built-in module for handling file paths and directories         |


### 1.5 Verifying the Installation

To make sure everything is installed correctly, create and run the following test script:

In [ ]:
# test_installation.py

import os
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, filters, morphology, measure
from scipy.spatial import ConvexHull

print("✅ All libraries imported successfully!")

Run it with:

In [ ]:
python test_installation.py

You should see:

In [ ]:
✅ All libraries imported successfully!


### 1.6 Common Installation Issues

| ❌ Error                                     | 🔧 Solution                               |
|--------------------------------------------|-------------------------------------------|
| ModuleNotFoundError: No module named 'cv2' | Run `pip install opencv-python`           |
| No module named 'skimage'                  | Run `pip install scikit-image`            |
| Permission denied or Command not found     | Try using `python3` and check for typos   |
| Excel export issues                        | Ensure `openpyxl` is installed            |


---
### 🖼️ 1.7: Creating TIFF Images for Analysis"

In this chapter, you'll learn how to create or export TIFF images that are compatible with your Apical-out cell classifier.
We'll walk through using **ImageJ (Fiji)**, Python, and other tools.

### 🧪 Option 1: Using ImageJ / Fiji

[Fiji](https://imagej.net/software/fiji/) is a powerful, open-source tool for bioimage analysis.

#### ✅ Export TIFF from an existing image:
1. Open Fiji
2. `File → Open...` to load your image
3. Convert to grayscale if needed: `Image → Type → 8-bit`
4. `File → Save As → Tiff...` → save into your `data/` folder

#### 🧫 Create a new synthetic TIFF:
1. `File → New → Image...`
2. Choose `8-bit`, 512×512, black or white background
3. Draw cells using tools like Brush, Oval, etc.
4. `Save As → Tiff...`

### 🧬 Option 2: Create TIFF in Python

Use Python to generate synthetic data:

In [ ]:
python
import numpy as np
from skimage import io

# Create random grayscale image
image = np.random.rand(512, 512) * 255
image = image.astype(np.uint8)

# Save to TIFF
io.imsave("data/fake_image.tif", image)

### 🖼 Option 3: Export from Photoshop

You can use Adobe Photoshop to export microscope-like images in TIFF format.

### Steps:

1. **Convert to Grayscale**  
   Go to: `Image → Mode → Grayscale`

2. **Save as TIFF**  
   Go to: `File → Save As → TIFF`

3. **Disable Compression**  
   In the TIFF save dialog, uncheck any compression options.

4. **Save to `data/` Folder**  
   For compatibility with the pipeline, store the `.tif` image inside the project's `data/` directory.
"""


### 📦 Best Practices for Your TIFF Files

| Requirement   | Description                                |
|---------------|--------------------------------------------|
| Format        | `.tif` or `.tiff`                          |
| Channels      | Grayscale (1-channel) preferred            |
| Bit Depth     | 8-bit or 16-bit                            |
| Resolution    | 256×256 to 1024×1024                       |
| Folder        | Place images in the `data/` directory      |
| Naming        | Avoid spaces. Use `snake_case` if possible |

## 💻Chapter 3: Customizing the Analysis

This section shows how to adapt the pipeline for your own dataset and preferences. You can configure input images, tune detection sensitivity, change visualization styles, and customize output paths.

---

### 🔧 3.1 Setting Your Image Files

To analyze your own images, open `parameters.py` and edit the `image_files` dictionary to point to your `.tif` files:

```python
# -------- Define Image Files --------
image_files = {
    "Sample_A": "path/to/your/images/Sample_A.tif",
    "Sample_B": "path/to/your/images/Sample_B.tif",
    "Sample_C": "path/to/your/images/Sample_C.tif",
    "Sample_D": "path/to/your/images/Sample_D.tif",
}
```

✅ You may use descriptive names as keys, and ensure that the file paths are correct and accessible.

---

### 🔧 3.2 Adjusting Detection Parameters

To fine-tune the analysis, edit the following parameters in `parameters.py`:

```python
# -------- Parameters --------
min_cell_areas = [100, 50, 30, 15, 10, 5, 3]  # Try these thresholds in order
high_intensity_fraction = 0.8                # Fraction of max intensity for high-intensity pixel selection
distance_ratio_threshold = 0.4               # Distance ratio for Apical-out classification
eccentricity_threshold = 0.4                 # Filters out near-circular regions
```

💡 **Tips**:
- Use larger values in `min_cell_areas` to exclude small debris.
- Adjust `high_intensity_fraction` for brighter/dimmer images.
- Tune `distance_ratio_threshold` to change how strictly cells are classified as Apical-out.

---

### 🎨 3.3 Customizing Visualization Colors

You can change how results are displayed in the output image. By default:
- Apical-out cells are **blue**.
- The convex hull boundary is **yellow**.

In `image_processing.py`, these lines control the colors:

```python
cell_color = (0, 0, 255)   # RGB blue
...
ax.plot(..., color='yellow', linewidth=2)
```

You can customize them. For example:
```python
cell_color = (255, 0, 0)   # Red
...
ax.plot(..., color='lime', linewidth=2)     # Bright green hull
```

📘 Use RGB values or supported color names (like `'cyan'`, `'magenta'`, `'orange'`).

---

### 💾 3.4 Specifying Output Location and File Name

The analysis results are saved as an Excel file. In `main.py`, the default location is the Desktop:

```python
output_path = os.path.expanduser("~/Desktop")
output_excel_path = os.path.join(output_path, "cell_analysis.xlsx")
```

To change this, simply replace the path:

```python
output_path = "results/"
output_excel_path = os.path.join(output_path, "my_custom_results.xlsx")
```

📁 Make sure the folder exists before running the script, or you can create it programmatically with:

```python
os.makedirs(output_path, exist_ok=True)
```

---

### 📦 Notebook Cell: Interactive Configuration Example

In [ ]:
# Run this cell to configure your analysis interactively

import os
import ipywidgets as widgets
from IPython.display import display

# -------- Widgets --------
output_path_widget = widgets.Text(
    value='results/',
    placeholder='Enter output folder path',
    description='Output Path:',
    layout=widgets.Layout(width='500px')
)

file_name_widget = widgets.Text(
    value='cell_analysis.xlsx',
    placeholder='Enter output file name',
    description='File Name:',
    layout=widgets.Layout(width='400px')
)

min_area_widget = widgets.Text(
    value='100, 50, 30, 15, 10, 5, 3',
    placeholder='Comma-separated values',
    description='Min Cell Areas:',
    layout=widgets.Layout(width='500px')
)

intensity_frac_widget = widgets.FloatSlider(
    value=0.8,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Intensity Fraction:',
    readout_format='.2f'
)

distance_thresh_widget = widgets.FloatSlider(
    value=0.4,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Distance Ratio:',
    readout_format='.2f'
)

eccentricity_widget = widgets.FloatSlider(
    value=0.4,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Eccentricity:',
    readout_format='.2f'
)

color_widget = widgets.Dropdown(
    options=['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'lime'],
    value='blue',
    description='Cell Color:'
)

hull_color_widget = widgets.Dropdown(
    options=['yellow', 'white', 'orange', 'lime', 'purple'],
    value='yellow',
    description='Hull Color:'
)

# -------- Display Widgets --------
display(output_path_widget, file_name_widget,
        min_area_widget, intensity_frac_widget,
        distance_thresh_widget, eccentricity_widget,
        color_widget, hull_color_widget)
